<a href="https://colab.research.google.com/github/JF11579/A-Year-s-Worth-of-Google-Searches/blob/master/SameHouse/SP500_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get the  towns into 3 groups.  Get avg sale price for each group.

### Make a multi line chart with one line of the SP500 and the other 3 of the Monthly avg of the 3 town groups.



# Library


In [1]:
#!pip install pandas
#!pip3 install --upgrade pip
#!pip install plotly
#!pip install scikit-learn

In [2]:
import pandas as pd
from google.colab import drive  # <<<<<<<<<------
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

# OPTIONS

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# The Plan

## Part One: The houses

### Devide the house into Least, Avg, Most.

# Plot Line the 3 groups

## Part Two

### Make Line plot of the SP500

# Part Three: Merge the two plots


# Part One: the Houses

## House Data

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Real_Estate_Data

Mounted at /content/drive
/content/drive/MyDrive/DATA/Real_Estate_Data


In [33]:
houses = pd.read_csv('Combined_data_Jan_10_3.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Combined_data_Jan_10_3.csv'

## Pre-Processing houses

In [34]:
data = houses
data = data.dropna(subset=["Sold_Price"]).query("`Sold_Price` != 0")
data = data[['Address', 'Town','Zip','State','Style',
                    'ML#','Sold_Price','Baths_Full', 'Baths_Half',
                    'Bath','Bedrooms',
                      'Appearance','REO' , 'School_District_Name',
                 'Approx_Int_Square_Footage','Street_Name','Contract_Date', 'Lot_Square_Footage', 'Lot_Size']]

### Convert the date

In [35]:
# Convert 'Contract_Date' to datetime, coerce errors to NaT
data['Contract_Date'] = pd.to_datetime(data['Contract_Date'], errors='coerce')

# Extract the year
data['Year'] = data['Contract_Date'].dt.year

# Drop rows where 'Year' is NaN (including where 'Contract_Date' conversion failed)
data = data.dropna(subset=['Year'])

data['Year'] = data['Year'].astype(int)

#
data["Contract_Date"] = data["Contract_Date"].dt.strftime("%Y-%m-%d")

data["Month_Year"] = data["Contract_Date"].str[:7]

### consolidate  house styles

In [36]:
# Calculate the frequency of each style
style_counts = houses['Style'].value_counts(normalize=True) * 100

# Identify styles that are less than 5%
styles_to_replace = style_counts[style_counts < 5].index
#
# Replace styles that are less than 5% with 'Other'
houses['Style'] = houses['Style'].replace(styles_to_replace, 'Other')

In [37]:
houses['Town'].unique()

array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead', 'Freeport',
       'Roosevelt', 'Westbury', 'Port Washington'], dtype=object)

## Next devide houses into groups

In [38]:
# prompt: filter town to just the 9 mentioned below

towns = ['Roosevelt', 'Hempstead', 'Freeport', 'Farmingdale', 'Westbury',
         'Mineola', 'Locust Valley', 'Port Washington', 'Roslynss']
data = data[data['Town'].isin(towns)]


In [39]:
houses= data

In [40]:
data.head()

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.00,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.00,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11
2,19 White Spots Lane,Locust Valley,11560,NY,Exp Cape,2427641,650000.0,2.0,1.0,2.50,3.0,Mint,N,Locust Valley,2353.0,White Spots,2011-10-08,NaN,NaN,2011,2011-10
3,5 Quarry Road,Locust Valley,11560,NY,Exp Ranch,2396870,800000.0,3.0,0.0,3.00,4.0,Excellent,N,Locust Valley,2151.0,Quarry,2012-05-03,NaN,NaN,2012,2012-05
4,3 Nassau Road,Locust Valley,11560,NY,Colonial,2030725,467500.0,2.0,2.0,2.55,4.0,Diamond!!,N,Locust Valley,NaN,Nassau,2008-06-22,NaN,NaN,2008,2008-06


In [41]:
# prompt: my "houses" has a column called Town.  Town has 9 unique entries.  in the end I want to  make a plotly  multi line plot  where one line is called LEast and it shows the Year_Month  average of Town["Roosevelt','Hempstead' and 'Freeport'.  Then do the same for a Line  called Middle for towns=  Farmingdale, Mineola, and Westbury.  then lastly the same for the wons of Locust Valley, Port Washington, a

# ### Town Groups
# Define the groups
least_towns = ['Roosevelt', 'Hempstead', 'Freeport']
middle_towns = ['Farmingdale', 'Mineola', 'Westbury']
most_towns = ['Locust Valley', 'Port Washington', 'Glen Cove']

# Create a new column with the town groups
data['Town_Group'] = np.select(
    [data['Town'].isin(least_towns), data['Town'].isin(middle_towns), data['Town'].isin(most_towns)],
    ['Least', 'Middle', 'Most'],
    default='Other'
)

# Calculate the average sale price for each town group and month
avg_sale_price = data.groupby(['Month_Year', 'Town_Group'])['Sold_Price'].mean().unstack()

# ## Plot the average sale price for each town group
fig = px.line(avg_sale_price, title='Average Sale Price by Town Group')
fig.show()


## Now the same plot but utilizing normalized data

In [42]:
data.head()

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Month_Year,Town_Group
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.00,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,NaN,NaN,2006,2006-09,Most
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.00,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,NaN,NaN,2015,2015-11,Most
2,19 White Spots Lane,Locust Valley,11560,NY,Exp Cape,2427641,650000.0,2.0,1.0,2.50,3.0,Mint,N,Locust Valley,2353.0,White Spots,2011-10-08,NaN,NaN,2011,2011-10,Most
3,5 Quarry Road,Locust Valley,11560,NY,Exp Ranch,2396870,800000.0,3.0,0.0,3.00,4.0,Excellent,N,Locust Valley,2151.0,Quarry,2012-05-03,NaN,NaN,2012,2012-05,Most
4,3 Nassau Road,Locust Valley,11560,NY,Colonial,2030725,467500.0,2.0,2.0,2.55,4.0,Diamond!!,N,Locust Valley,NaN,Nassau,2008-06-22,NaN,NaN,2008,2008-06,Most


In [43]:
a = b

NameError: name 'b' is not defined

In [44]:
# Initialize the MinMaxScaler to scale between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler to your data and transform it
data['Normalized_Sold_Price'] = scaler.fit_transform(data[['Sold_Price']])


In [ ]:
# prompt: create new column called Normalized_Sold_Price based on Sold_Price

#data['Normalized_Sold_Price'] = (data['Sold_Price'] - data['Sold_Price'].min()) / (data['Sold_Price'].max() - data['Sold_Price'].min())


In [ ]:
data.info()

## Now plot using the normailzed sold Price

In [45]:
import pandas as pd
import numpy as np
import plotly.express as px

# Assume 'data' is your DataFrame that contains all the necessary columns
# And assume 'Normalized_Sold_Price' is already present in 'data'

# Define the groups
least_towns = ['Roosevelt', 'Hempstead', 'Freeport']
middle_towns = ['Farmingdale', 'Mineola', 'Westbury']
most_towns = ['Locust Valley', 'Port Washington', 'Glen Cove']

# Create a new column with the town groups
data['Town_Group'] = np.select(
    [data['Town'].isin(least_towns), data['Town'].isin(middle_towns), data['Town'].isin(most_towns)],
    ['Least', 'Middle', 'Most'],
    default='Other'
)

# Calculate the average normalized sale price for each town group and month
avg_normalized_sale_price = data.groupby(['Month_Year', 'Town_Group'])['Normalized_Sold_Price'].mean().unstack()

# Plot the average normalized sale price for each town group
fig = px.line(avg_normalized_sale_price, title='Average Normalized Sale Price by Town Group')

# Adjusting the layout if necessary, like changing y-axis title
fig.update_layout(yaxis_title='Average Normalized Sale Price')

# Show the plot
fig.show()


## Now add in the SP500 plot

## SP500 Data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Uncatergorized_Data

In [ ]:
SP500 = pd.read_csv('S&P500_yahoo.csv')

## Create Monthly adjusted mean

In [ ]:
SP500['Date'] = pd.to_datetime(SP500['Date'])

# Now, create the 'Month_Year' column formatted as "Month YYYY"
SP500['Month_Year'] = SP500['Date'].dt.strftime('%b %Y')

# Show the first few rows to verify
SP500.head()

In [ ]:
SP500['Adj Close**'] = SP500['Adj Close**'].str.replace(',', '').astype(float)

In [ ]:
# Convert 'Adj Close**' to numeric, forcing non-numeric values to NaN (errors='coerce')
SP500['Adj Close**'] = pd.to_numeric(SP500['Adj Close**'], errors='coerce')

# Group by 'Month_Year' and calculate the mean of 'Adj Close**'
mean_adj_close_by_month_year = SP500.groupby('Month_Year')['Adj Close**'].mean()

# Reset index if you want the result as a DataFrame
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year.reset_index()

# Show the result
#print(mean_adj_close_by_month_year_df)
mean_adj_close_by_month_year_df.head(5)

## normalize

In [ ]:
# Assume mean_adj_close_by_month_year_df is your DataFrame and 'Adj Close**' is the column to be normalized

# Calculating Min and Max values
min_value = mean_adj_close_by_month_year_df['Adj Close**'].min()
max_value = mean_adj_close_by_month_year_df['Adj Close**'].max()

# Applying the normalization formula
mean_adj_close_by_month_year_df['NormalizedAdjClose'] = (mean_adj_close_by_month_year_df['Adj Close**'] - min_value) / (max_value - min_value)

# Display the first few rows to verify the new column
mean_adj_close_by_month_year_df.head(3)

## Make sure in chrono order

In [ ]:
# prompt: lets make sure that the df "mean_adj_close_by_month_year_df" has both only one row per month and is in chronological order

# Sort the DataFrame by 'Month_Year'
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.sort_values(by='Month_Year')

# Drop duplicate rows based on 'Month_Year'
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.drop_duplicates(subset='Month_Year', keep='last')

# Reset the index to ensure consecutive indexing
mean_adj_close_by_month_year_df = mean_adj_close_by_month_year_df.reset_index(drop=True)

# Now, mean_adj_close_by_month_year_df has one row per month in chronological order

now lets have test plot

In [ ]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for Product A and Product B
fig.add_trace(go.Scatter(x= mean_adj_close_by_month_year_df['Month_Year'], y= mean_adj_close_by_month_year_df['NormalizedAdjClose'], mode='lines+markers', name='Houses'))
#fig.add_trace(go.Scatter(x=DF_select_sorted['Month_Year'], y=DF['NormalizedAdjClose'], mode='lines+markers', name='SP 500'))

# Add figure title and labels
fig.update_layout(title='Sales of Product A and B Over Time',
                   xaxis_title='Month_Year',
                   yaxis_title='Sales',
                   template='plotly_dark') # Using a dark theme for the plot

# Show plot
fig.show()

In [ ]:
##############

In [ ]:
# Now Merge house and meadn adj close

import pandas as pd

# Convert 'Month_Year' to datetime
mean_adj_close_by_month_year_df['Month_Year'] = pd.to_datetime(mean_adj_close_by_month_year_df['Month_Year'], format='%b %Y')

# Format 'Month_Year' to "YYYY-MM"
mean_adj_close_by_month_year_df['Month_Year'] = mean_adj_close_by_month_year_df['Month_Year'].dt.strftime('%Y-%m')

# Now, both DataFrames should have the 'Month_Year' column formatted as "YYYY-MM", and you can merge them.
DF = pd.merge(houses, mean_adj_close_by_month_year_df, on='Month_Year', how='inner')

# Display the first few rows of the merged DataFrame
DF.head(3)

In [ ]:
#multiple_columns = df[['ColumnName1', 'ColumnName2']]
DF_select = DF[['Month_Year', 'Normalized_All Sold Avg Per Month' , 'NormalizedAdjClose']]

In [ ]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for Product A and Product B
fig.add_trace(go.Scatter(x= DF_select_sorted['Month_Year'], y=DF['Normalized_All Sold Avg Per Month'], mode='lines+markers', name='Houses'))
fig.add_trace(go.Scatter(x=DF_select_sorted['Month_Year'], y=DF['NormalizedAdjClose'], mode='lines+markers', name='SP 500'))

# Add figure title and labels
fig.update_layout(title='Sales of Product A and B Over Time',
                   xaxis_title='Month_Year',
                   yaxis_title='Sales',
                   template='plotly_dark') # Using a dark theme for the plot

# Show plot
fig.show()

